In [13]:
#coding=utf-8
import numpy as np
import PyKDL as kdl
import xml.dom.minidom
from urdf_parser_py.urdf import URDF
# from pykdl_utils.kdl_parser import kdl_tree_from_urdf_model
import os
import numpy as np
import copy

In [2]:
def get_urdf_parameter(joint_id,xml_obj):
    """ get inertial data. """
    # mass = xml_obj.getElementsByTagName("mass")[joint_id].getAttribute("value")
    rpy  = xml_obj.getElementsByTagName("joint")[joint_id].getElementsByTagName("origin")[0].getAttribute("rpy").split(" ")
    rpy  = tuple([float(i) for i in rpy])
    axis  = xml_obj.getElementsByTagName("joint")[joint_id].getElementsByTagName("axis")[0].getAttribute("xyz").split(" ")
    axis  = np.array(axis,dtype=np.float32)
    xyz  = xml_obj.getElementsByTagName("joint")[joint_id].getElementsByTagName("origin")[0].getAttribute("xyz").split(" ")
    xyz  = np.array(xyz,dtype=np.float32)
    data = [axis, rpy,xyz]
    return data

In [35]:
import PyKDL as kdl
from urdf_parser_py.urdf import Robot

def euler_to_quat(r, p, y):
    sr, sp, sy = np.sin(r/2.0), np.sin(p/2.0), np.sin(y/2.0)
    cr, cp, cy = np.cos(r/2.0), np.cos(p/2.0), np.cos(y/2.0)
    return [sr*cp*cy - cr*sp*sy,
            cr*sp*cy + sr*cp*sy,
            cr*cp*sy - sr*sp*cy,
            cr*cp*cy + sr*sp*sy]

def urdf_pose_to_kdl_frame(pose):
    pos = [0., 0., 0.]
    rot = [0., 0., 0.]
    if pose is not None:
        if pose.position is not None:
            pos = pose.position
        if pose.rotation is not None:
            rot = pose.rotation
    return kdl.Frame(kdl.Rotation.Quaternion(*euler_to_quat(*rot)),
                     kdl.Vector(*pos))

def urdf_joint_to_kdl_joint(jnt):
    origin_frame = urdf_pose_to_kdl_frame(jnt.origin)
    if jnt.joint_type == 'fixed':
        return kdl.Joint(jnt.name, kdl.Joint.Fixed)
    axis = kdl.Vector(*jnt.axis)
    if jnt.joint_type == 'revolute':
        return kdl.Joint(jnt.name, origin_frame.p,
                         origin_frame.M * axis, kdl.Joint.RotAxis)
    if jnt.joint_type == 'continuous':
        return kdl.Joint(jnt.name, origin_frame.p,
                         origin_frame.M * axis, kdl.Joint.RotAxis)
    if jnt.joint_type == 'prismatic':
        return kdl.Joint(jnt.name, origin_frame.p,
                         origin_frame.M * axis, kdl.Joint.TransAxis)
    # prin "Unknown joint type: %s." % jnt.joint_type
    return kdl.Joint(jnt.name, kdl.Joint.Fixed)

def urdf_inertial_to_kdl_rbi(i):
    origin = urdf_pose_to_kdl_frame(i.origin)
    rbi = kdl.RigidBodyInertia(i.mass, origin.p,
                               kdl.RotationalInertia(i.inertia.ixx,
                                                     i.inertia.iyy,
                                                     i.inertia.izz,
                                                     i.inertia.ixy,
                                                     i.inertia.ixz,
                                                     i.inertia.iyz))
    return origin.M * rbi

##
# Returns a PyKDL.Tree generated from a urdf_parser_py.urdf.URDF object.
def kdl_tree_from_urdf_model(urdf):
    root = urdf.get_root()
    tree = kdl.Tree(root)
    def add_children_to_tree(parent):
        if parent in urdf.child_map:
            for joint, child_name in urdf.child_map[parent]:
                child = urdf.link_map[child_name]
                if child.inertial is not None:
                    kdl_inert = urdf_inertial_to_kdl_rbi(child.inertial)
                else:
                    kdl_inert = kdl.RigidBodyInertia()
                kdl_jnt = urdf_joint_to_kdl_joint(urdf.joint_map[joint])
                kdl_origin = urdf_pose_to_kdl_frame(urdf.joint_map[joint].origin)
                kdl_sgm = kdl.Segment(child_name, kdl_jnt,
                                      kdl_origin, kdl_inert)
                tree.addSegment(kdl_sgm, parent)
                add_children_to_tree(child_name)
    add_children_to_tree(root)
    return tree

In [36]:
robot = URDF.from_xml_file("dofbot.urdf")

In [38]:
xml_string = robot.to_xml_string()
print(xml_string)
tree = kdl_tree_from_urdf_model(robot)

<?xml version="1.0" ?>
<robot name="dofbot" version="1.0">
  <link name="base_link">
    <visual>
      <origin xyz="0.0 0.0 0.0" rpy="0.0 0.0 0.0"/>
      <geometry>
        <mesh filename="package://dofbot_moveit/meshes/base_link.STL"/>
      </geometry>
      <material name="">
        <color rgba="0.7 0.7 0.7 1.0"/>
      </material>
    </visual>
    <collision>
      <origin xyz="0.0 0.0 0.0" rpy="0.0 0.0 0.0"/>
      <geometry>
        <mesh filename="package://dofbot_moveit/meshes/base_link.STL"/>
      </geometry>
    </collision>
  </link>
  <link name="link1">
    <visual>
      <origin xyz="0.0 0.0 0.0" rpy="0.0 0.0 0.0"/>
      <geometry>
        <mesh filename="package://dofbot_moveit/meshes/link1.STL"/>
      </geometry>
      <material name="">
        <color rgba="0.0 0.7 0.0 1.0"/>
      </material>
    </visual>
    <collision>
      <origin xyz="0.0 0.0 0.0" rpy="0.0 0.0 0.0"/>
      <geometry>
        <mesh filename="package://dofbot_moveit/meshes/link1.STL"/>
    

In [39]:
tree

base_link(q_nr: 0)
 	link1(q_nr: 0)
 	link2(q_nr: 1)
 	link3(q_nr: 2)
 	link4(q_nr: 3)
 	link5(q_nr: 4)
 	
	
	
	
	
	

In [349]:
chain = tree.getChain("base_link", "link5")
fk_solver = kdl.ChainFkSolverPos_recursive(chain)
number_of_joints = chain.getNrOfJoints()
q = kdl.JntArray(5)
q[1] = np.pi/3
q[2]  = np.pi/3
# Create a frame to hold the end effector pose result
end_effector_pose = kdl.Frame()

# Calculate forward kinematics
fk_solver.JntToCart(q, end_effector_pose)
# Create the inverse kinematics solver
ik_solver = kdl.ChainIkSolverPos_LMA(chain,eps=1e-3,maxiter=10000)

# Desired end-effector pose

desired_pose = kdl.Frame(kdl.Rotation.RPY(0, 0, 0), kdl.Vector(0.11, 0.02, 0.12))
# Solution joint positions will be stored here
solution = kdl.JntArray(number_of_joints)
q_int = kdl.JntArray(5)
q_int[0]  = -np.arctan2(desired_pose.p[0], desired_pose.p[1])
# Calculate inverse kinematics
status = ik_solver.CartToJnt(q_int, desired_pose, solution)
# while status != 0:
#     q_int[1]  = np.random.uniform(low=-np.pi/2, high=np.pi/2)
#     q_int[2]  = np.random.uniform(low=-np.pi/2, high=np.pi/2)
#     q_int[3]  = np.random.uniform(low=-np.pi/2, high=np.pi/2)
#     status = ik_solver.CartToJnt(q_int, desired_pose, solution)
print(status)
if status >= 0:
    print("Solution found!") 
    # `solution` now contains the joint positions
else:
    print("No Solution")
    
print(solution)

0
Solution found!
[    -1.39094    -1.38649    -1.36212     2.73928     1.39081]


In [350]:
desired_pose 

[[           1,           0,           0;
            0,           1,           0;
           -0,           0,           1]
[        0.11,        0.02,        0.12]]

In [351]:
fk_solver.JntToCart(solution, end_effector_pose )
end_effector_pose 

[[    0.999958,  0.00012749,  0.00918266;
 -0.000142828,    0.999999,  0.00166959;
  -0.00918243, -0.00167083,    0.999956]
[    0.109911,    0.019984,    0.120016]]

In [387]:
q = kdl.JntArray(5)
q[0] = 0
q[1]  = -np.pi/2
# Create a frame to hold the end effector pose result
end_effector_pose = kdl.Frame()

# Calculate forward kinematics
fk_solver.JntToCart(q, end_effector_pose)

0

In [388]:
end_effector_pose

[[           1,-3.67321e-06,-3.67321e-06;
  3.67321e-06, 6.12323e-17,           1;
 -3.67321e-06,          -1, 1.34925e-11]
[ 7.89739e-09,     0.23955,     0.10965]]

In [395]:
# Desired end-effector pose

desired_pose = kdl.Frame(kdl.Rotation.RPY(-np.pi/2, np.pi/2, 0), kdl.Vector(0., 0.2, 0.05))
# Solution joint positions will be stored here
solution = kdl.JntArray(number_of_joints)
q_int = kdl.JntArray(5)
q_int[0]  = -np.arctan2(desired_pose.p[0], desired_pose.p[1])
# Calculate inverse kinematics
status = ik_solver.CartToJnt(q_int, desired_pose, solution)
# while status != 0:
q_int[1]  = np.random.uniform(low=0, high=np.pi)
q_int[2]  = np.random.uniform(low=0, high=np.pi)
q_int[3]  = np.random.uniform(low=0, high=np.pi)
q_int[4]  = np.random.uniform(low=0, high=np.pi)
#     status = ik_solver.CartToJnt(q_int, desired_pose, solution)
print(status)
if status >= 0:
    print("Solution found!") 
    # `solution` now contains the joint positions
else:
    print("No Solution")
fk_solver.JntToCart(solution, end_effector_pose)
print(end_effector_pose)
print(desired_pose)

0
Solution found!
[[ 0.000530733,          -1,-2.96503e-06;
   -0.0144023,-1.06085e-05,    0.999896;
    -0.999896,-0.000530636,  -0.0144023]
[-5.91143e-08,    0.199823,   0.0499994]]
[[ 6.12323e-17,          -1, 6.12323e-17;
            0, 6.12323e-17,           1;
           -1,-6.12323e-17,  3.7494e-33]
[           0,         0.2,        0.05]]


In [396]:
solution
print(solution[0],",",solution[1],",",solution[2],",",solution[3],",",solution[4])

-7.611595525802034e-07 , -1.4295786783796354 , -1.1532132390028922 , 0.9975928272716723 , 1.570261909573396


In [391]:
# Desired end-effector pose

desired_pose = kdl.Frame(kdl.Rotation.EulerZYX(0, np.pi/2, np.pi/2), kdl.Vector(0.3, 0., 0.05))
# Solution joint positions will be stored here
solution = kdl.JntArray(number_of_joints)
q_int = kdl.JntArray(5)
q_int[0]  = -np.arctan2(desired_pose.p[0], desired_pose.p[1])
# Calculate inverse kinematics
status = ik_solver.CartToJnt(q_int, desired_pose, solution)
while status != 0:
    q_int[1]  = np.random.uniform(low=-np.pi/2, high=np.pi/2)
    q_int[2]  = np.random.uniform(low=-np.pi/2, high=np.pi/2)
    q_int[3]  = np.random.uniform(low=-np.pi/2, high=np.pi/2)
    status = ik_solver.CartToJnt(q_int, desired_pose, solution)
print(status)
if status >= 0:
    print("Solution found!") 
    # `solution` now contains the joint positions
else:
    print("No Solution")
    
print(solution)

KeyboardInterrupt: 

[   -0.463648    -1.62649     -1.1286     2.75507    0.463648]

SyntaxError: invalid syntax (159487375.py, line 1)